In [328]:
import numpy as np
import pandas as pd
from transaction_dates import *
from imiona import *
from collections import Counter
from samochody import *
from czesci import *

# Pracownicy

In [329]:
birth_dates_lims = [1960, 1995]
begining_date = "2014-01-01"
end_date = "2018-01-01"

def rind(a, b):
    return np.random.randint(a, b)

jobs_list = np.array(["menadżer", "sprzedawca", "mechanik", "pomocnik mechanika", 
                       "diagnosta samochodowy", "sprzątający", "informatyk", "księgowy"])
jobs_amount = np.array([1, rind(2, 5), rind(3, 6), rind(1, 3),
                        rind(1, 3), rind(1, 3), rind(1, 3), rind(0, 2)])
fixed_jobs = np.array([0, 1, 2, np.sum(jobs_amount[:2]), np.sum(jobs_amount[0:6])]) # zaczynamy z szefem, dwoma sprzedawcami i mechanikiem

employment_dates = random_transaction_dates(begining_date, end_date, np.sum(jobs_amount))
employment_dates[fixed_jobs] = begining_date  

wages_lims = [(60, 75), (45, 60), (45, 60), (35, 45),
              (47, 60), (35, 40), (40, 60), (40, 53)]

employees_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "pesel",
                                     "miasto", "adres", "telefon", "email", "data_zatrudnienia",
                                     "data_zwolnienia", "stanowisko", "płaca[zł/h]"])
dismissal_dates = [[] for i in range(len(jobs_list))]
for (i, job) in enumerate(jobs_list):

    begin_times = ["" for j in range(jobs_amount[i])]
    for j in range(jobs_amount[i]):
        employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
        if np.random.rand() <= 0.5:
            first_name, last_name = random_name_man()
            pesel, birth_date = random_pesel_date_man(birth_dates_lims)
            sex = "M"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        else:
            first_name, last_name = random_name_woman()
            pesel, birth_date = random_pesel_date_woman(birth_dates_lims)
            sex = "K"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        begin_times[j] = employment_date
        if np.random.rand() <= (jobs_amount[i]-1)/10 * 1/(1+c):
            dismissal_date = random_transaction_dates(str(employment_date), end_date, 1)[0]
            c += 1
        else:
            dismissal_date = None

        employees_df.loc[np.sum(jobs_amount[:i]) + j] = [0, first_name, last_name, pesel,
                                                         city, address, phone, email, employment_date,
                                                         dismissal_date, job, wage]
    dismissal_dates_j = [None for j in range(jobs_amount[i])]
    c = 0
    for j in range(len(begin_times)):
        if len(begin_times) >= 2:
            min_date = min(begin_times[:j] + begin_times[j+1:])
        else:
            min_date = begin_times[j]
        if np.random.rand() <= (jobs_amount[i]-1)/10 * 1/(1+c):
            dismissal_date = random_transaction_dates(str(max(min_date, begin_times[j])), end_date, 1)[0]
            okay = False
            for k in range(len(dismissal_dates_j)):
                if (dismissal_dates_j[k] is not None) and (dismissal_date >=  begin_times[k]) and (dismissal_date <= dismissal_dates_j[k]):
                    okay = True
                    break
                elif dismissal_date >=  begin_times[k]:
                    okay = True
                    break
            if okay:
                dismissal_dates_j[j] = dismissal_date
                c += 1
    dismissal_dates[i] = dismissal_dates_j

diss = []
for i in range(len(dismissal_dates)):
    diss += dismissal_dates[i]
employees_df["data_zwolnienia"] = diss

employees_df = employees_df.sort_values(by="data_zatrudnienia")   
employees_df["id"] =  np.arange(1, len(employees_df)+1)

In [330]:
employees_df

,id,imię,nazwisko,pesel,miasto,adres,telefon,email,data_zatrudnienia,data_zwolnienia,stanowisko,płaca[zł/h]
0,1,Jolanta,Hulchevska,65042627464,Wrocław,ul. Marii Curie-Skłodowskiej 21/37,48623480243,jolanta.hulchevska@kuba.pl,2014-01-01,None,menadżer,61.2
1,2,Grażyna,Rawłuszko,87122148643,Wrocław,ul. mjr. Jana Piwnika-Ponurego 72/20,48636732778,grazyna.rawluszko@kuba.pl,2014-01-01,2015-02-21,sprzedawca,47.7
2,3,Sebastian,Rozwadowski,73121089711,Wrocław,ul. Polna 53,48495740796,sebastian.rozwadowski@kuba.pl,2014-01-01,None,sprzedawca,49.1
4,4,Elżbieta,Bida,62092206548,Wrocław,ul. 1 Maja 16/50,48433131244,elzbieta.bida@kuba.pl,2014-01-01,None,mechanik,54.3
13,5,Dorota,Kozińska,84010540442,Wrocław,ul. Jagodowa 52/46,48273232515,dorota.kozinska@kuba.pl,2014-01-01,None,informatyk,56.9
3,6,Piotr,Cywiński,84080857914,Drezdenko,ul. Makowska 97,48193551260,piotr.cywinski@kuba.pl,2014-11-30,None,sprzedawca,52.7
5,7,Lena,Sypka,90071165502,Gryfice,ul. Leśna 9,48052694372,lena.sypka@kuba.pl,2015-08-29,None,mechanik,47.8
6,8,Maria,Chmiel,78010532624,Wrocław,ul. Brzozowa 20/6,48927665569,maria.chmiel@kuba.pl,2016-03-25,None,mechanik,56.9
7,9,Marek,Klapa,86080649210,Wrocław,ul. Leśna 1,48450316789,marek.klapa@kuba.pl,2016-05-05,None,mechanik,50.6
8,10,Barbara,Olender,74082185645,Wrocław,ul. Felicji Grunwaldowej 87,48407548217,barbara.olender@kuba.pl,2016-06-29,None,pomocnik mechanika,44.3


# Transakcje

In [331]:
no_transactions = 2000
begining_buy = 20

dates = random_transaction_dates("2014-01-01", "2020-12-31", no_transactions)

In [332]:
client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions

j = 1
for i in range(len(dates)):
    prob = i/(2*len(dates))
    if np.random.rand() <= prob:
        client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
    else:
        client_ids[i] = j
        j += 1

In [333]:
car_ids = np.zeros(len(dates), dtype=int) # array with car ids for table with transactions
car_ids[:begining_buy] = np.arange(1, 21)

client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions
client_ids[:begining_buy] = np.arange(1, 21)

transactions_types = np.zeros(len(dates))

clients_cars = {}
our_cars = np.arange(1, 21).tolist()

jp = 21
jc = 21
for i in range(20, len(dates)):
    weights = np.array([1.5*len(clients_cars)/max(1, len(our_cars)) , len(our_cars)*50, len(clients_cars)/(5*max(1, len(our_cars)))])
    weights = weights/np.sum(weights)
    transaction_type = np.random.choice(np.arange(3), p=weights) # 0-buy, 1-sell, 2-service
    transactions_types[i] = transaction_type
    if transaction_type == 0:
        prob = i/(2*len(dates)) 
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars and np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                clients_cars[client_ids[i]].remove(car_ids[i])
            else:
                car_ids[i] = jc
                jc += 1
            our_cars.append(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            our_cars.append(car_ids[i])
            jp += 1
            jc += 1

    if transaction_type == 1:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            car_ids[i] = np.random.choice(our_cars)
            if client_ids[i] in clients_cars:
                clients_cars[client_ids[i]].append(car_ids[i])
            else:
                clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = np.random.choice(our_cars)
            clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            jp += 1
    
    if transaction_type == 2:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars:
                if np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                    car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                else:
                    car_ids[i] = jc
                    clients_cars[client_ids[i]].append(car_ids[i])
                    jc += 1
            else:
                car_ids[i] = jc
                clients_cars[client_ids[i]] = [car_ids[i]]
                jc += 1
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            clients_cars[client_ids[i]] = [car_ids[i]]
            jp += 1
            jc += 1

In [334]:
buy_indexes = np.where(transactions_types == 0)
sell_indexes = np.where(transactions_types == 1)
service_indexes = np.where(transactions_types == 2)

In [335]:
transaction_dates_buy = dates[buy_indexes]
transaction_dates_sell = dates[sell_indexes]
transaction_dates_service = dates[service_indexes]

client_ids_buy = client_ids[buy_indexes]
client_ids_sell = client_ids[sell_indexes]
client_ids_service = client_ids[service_indexes]

car_ids_buy = car_ids[buy_indexes]
car_ids_sell = car_ids[sell_indexes]
car_ids_service = car_ids[service_indexes]

In [336]:
buy_employee_id = np.zeros(len(transaction_dates_buy))

for i in range(len(transaction_dates_buy)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_buy[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_buy[i]
    buy_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [337]:
sell_employee_id = np.zeros(len(transaction_dates_sell))

for i in range(len(transaction_dates_sell)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_sell[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_sell[i]
    sell_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [338]:
service_employee_id = np.zeros(len(transaction_dates_service))

for i in range(len(transaction_dates_service)):
    filter1 = employees_df["stanowisko"] == "mechanik"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_service[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_service[i]
    service_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])


# klienci

In [339]:
year_lims = [1970, 2010]
transaction_counter = Counter(client_ids)

clients_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "płeć", "PESEL", "miasto", "adres", "karta_lojalnościowa", "telefon", "email"])
unique_clients = np.unique(client_ids)
for i in range(len(unique_clients)):
    if np.random.rand() <= 0.5: # chłop probability
        id_ = unique_clients[i]
        first_name, last_name = random_name_man()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man(year_lims)
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_lims[0], year_lims[1]))
        sex = "M"
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
    else:
        id_ = unique_clients[i]
        first_name, last_name = random_name_woman()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man(year_lims)
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_lims[0], year_lims[1]))
        sex = "F"
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]

In [340]:
clients_df

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,1,Sebastian,Kleszcz,M,04300469354,Wrocław,ul. Lwowska 67/16,False,48158296030,f6uw1fo1y2i6wd0@opayq.com
1,2,Bogdan,Boschke,M,71010674237,Ożarów,ul. ks. Władysława Demskiego 69/10,False,48101553161,b_boschk1971@gmail.com
2,3,Antoni,Stokłosa,M,80011356035,Wrocław,ul. Kruszwicka 20,False,48214657387,a.stoklos@gmail.com
3,4,Dominik,Pietrzak,M,07251511077,Wrocław,ul. Polna 2,False,48626920655,d.pietrz@wp.pl
4,5,Nela,Setla,F,80081335059,Gniezno,ul. Wysoka 48/42,False,48552240603,setla19802@wp.pl
...,...,...,...,...,...,...,...,...,...,...
1507,1508,Olga,Orlikowska,F,99041450871,Wrocław,ul. Wyzwolenia 76/27,False,48423547232,olg-orlikow99@gmail.com
1508,1509,Gabriela,Jachowska,F,06282905019,Nowa Ruda,ul. Wypoczynkowa 89/34,False,48252338845,jachowska20065@interia.pl
1509,1510,Nadia,Skrzypek,F,87080528992,Kartuzy,ul. Bolechowskich 11/27,False,48278642607,n-skrzypek87@gazeta.pl
1510,1511,Jakub,Aghjayev,M,99022134530,Wrocław,ul. Niezapominajki 58/30,False,48466131340,kniqiz2gr8@opayq.com


In [341]:
np.sum(clients_df['karta_lojalnościowa'])

78

In [342]:
np.sum(pd.isna(clients_df["PESEL"]))

70

# auta

In [343]:
rng = np.random.default_rng()

data_car, data_bike = get_car_data()
make_list_car, model_list_car, car_weights = get_car_weights()
make_list_bike, model_list_bike, bike_weights = get_bike_weights()

In [344]:
def create_car(i):

    is_car = rng.random() < 0.9
    colors = np.array(('czarny', 'biały', 'szary', 'srebrny', 'niebieski', 'czerwony', 'brązowy', 'złoty', 'żółty', 'inny'))
    colors_weights = np.array((0.22, 0.19, 0.18, 0.15, 0.10, 0.09, 0.03, 0.02, 0.01, 0.01))

    if is_car:
        index = rng.choice(len(make_list_car), p=car_weights)
        make, model = make_list_car[index], model_list_car[index]
        weights = data_car[(data_car['make'] == make) & (data_car['model'] == model)]
        body = np.array(('SUV', 'Hatchback', 'Sedan', 'Kombi', 'Coupe', 'Van', 'Minivan', 'Kabriolet'))
        body_weights = np.array((0.25, 0.25, 0.23, 0.17, 0.06, 0.03, 0.009, 0.001))
    else:
        index = rng.choice(len(make_list_bike), p=bike_weights)
        make, model = make_list_bike[index], model_list_bike[index]
        weights = False
        body = False
        body_weights = False

    wheels = 4 if is_car else 2
    year = get_year()
    gear = get_gear(weights, is_car)
    fuel = get_fuel(weights, is_car)
    post_accident = get_post_accident()
    hp = get_hp(weights, is_car)
    color = get_color(colors, colors_weights)
    body_type = get_body_type(body, body_weights, is_car)
    engine_capacity = get_engine_capacity(hp, is_car)
    doors = get_doors(is_car)
    seats = get_num_of_seats(is_car)
    available = i+1 in our_cars
    
    return [i+1, make, model, wheels, year, body_type, post_accident, color, fuel, engine_capacity, hp, gear, doors, seats, available]


def create_car_table(n):
    
    df = pd.DataFrame(columns=['id', 'marka', 'model', 'liczba_kół', 'rok_produkcji', 'typ_nadwozia', 'czy_powypadkowy', 'kolor', 'typ_paliwa', 'pojemność_silnika', 'moc', 'skrzynia_biegów', 'liczba_drzwi', 'liczba_miejsc', 'czy_dostępny'])
    for i in range(n):
        row = create_car(i)
        df.loc[i] = row
    return df


In [345]:
df_samochody = create_car_table(max(car_ids))

In [346]:
df_samochody

,id,marka,model,liczba_kół,rok_produkcji,typ_nadwozia,czy_powypadkowy,kolor,typ_paliwa,pojemność_silnika,moc,skrzynia_biegów,liczba_drzwi,liczba_miejsc,czy_dostępny
0,1,smart,forFour,4,2004,Sedan,False,szary,Elektryczny,1.1,71,Manualna,5,5,False
1,2,Opel,Corsa,4,2006,Van,False,niebieski,Benzyna,1.4,90,Manualna,3,5,False
2,3,Fiat,500,4,2004,Minivan,False,czerwony,Benzyna,1.4,86,Manualna,5,5,False
3,4,Honda,CR-V,4,2003,Kombi,False,czarny,Benzyna,2.3,150,Automatyczna,5,5,False
4,5,Hyundai,i30,4,2000,Sedan,False,czarny,Benzyna,1.7,120,Manualna,5,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980,981,Audi,A1,4,2004,Hatchback,False,niebieski,Benzyna,1.5,105,Automatyczna,5,2,False
981,982,Audi,A4,4,2008,Sedan,False,czarny,Diesel,2.6,170,Manualna,5,5,True
982,983,Mitsubishi,Space Star,4,2011,Sedan,False,biały,Benzyna,1.0,71,Manualna,5,5,False
983,984,Volkswagen,T6 California,4,2007,SUV,False,biały,Diesel,2.4,150,Manualna,5,5,False


# Usługi

In [347]:
service_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'mechanik']['id'], len(car_ids_service))
service_ids = np.arange(len(transaction_dates_service)) + 1

df_services = pd.DataFrame()
df_services['id'] = service_ids
df_services['id_samochodu'] = car_ids_service
df_services['id_klienta'] = client_ids_service
df_services['id_pracownika'] = service_employee
df_services['data_zaksiegowania'] = transaction_dates_service

# Zakup samochodów

In [348]:
def car_purchase_price(hp, yr_bought, yr_prod, post_acc):
    return np.round(hp * 1000 / np.sqrt(int(yr_bought) - int(yr_prod)) * (0.75 if post_acc else 1) * rng.uniform(0.8, 1.2), -2)

In [349]:
buy_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_buy))
buy_ids = np.arange(len(transaction_dates_buy)) + 1

df_buy = pd.DataFrame()
df_buy['id'] = buy_ids
df_buy['id_samochodu'] = car_ids_buy
df_buy['id_klienta'] = client_ids_buy
df_buy['id_pracownika'] = buy_employee
df_buy['data_zaksiegowania'] = transaction_dates_buy

In [350]:

service_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_services[['id_samochodu', 'data_zaksiegowania']], how='right', left_on='id', right_on='id_samochodu')
service_date_fix['data_zaksiegowania'] = service_date_fix['data_zaksiegowania'].str[:4]

buy_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_buy[['id_samochodu', 'data_zaksiegowania']], how='right', left_on='id', right_on='id_samochodu')
buy_date_fix['data_zaksiegowania'] = buy_date_fix['data_zaksiegowania'].str[:4]

for i in range(len(buy_date_fix)):
    if str(buy_date_fix['rok_produkcji'].loc()[i]) >= str(buy_date_fix['data_zaksiegowania'].loc()[i]):
        car_index = buy_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

for i in range(len(service_date_fix)):
    if str(service_date_fix['rok_produkcji'].loc()[i]) >= str(service_date_fix['data_zaksiegowania'].loc()[i]):
        car_index = service_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

C:\Users\olkar\AppData\Local\Temp\ipykernel_17124\1855781291.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2006' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)


In [351]:
purchase_prices = []
for j, i in enumerate(car_ids_buy):
    row = df_samochody.loc()[i-1]
    year_bought = transaction_dates_buy[j][:4]
    purchase_prices.append(car_purchase_price(row['moc'], year_bought, row['rok_produkcji'], row['czy_powypadkowy']))

df_buy['cena'] = purchase_prices

# Sprzedaż samochodów

In [352]:
sell_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_sell))
sell_ids = np.arange(len(transaction_dates_sell)) + 1

df_sell = pd.DataFrame()
df_sell['id'] = sell_ids
df_sell['id_samochodu'] = car_ids_sell
df_sell['id_klienta'] = client_ids_sell
df_sell['id_pracownika'] = sell_employee
df_sell['data_zaksiegowania'] = transaction_dates_sell

In [353]:
df_sell

,id,id_samochodu,id_klienta,id_pracownika,data_zaksiegowania
0,1,18,21,2,2014-01-30
1,2,17,22,2,2014-01-30
2,3,9,23,6,2014-02-04
3,4,16,24,2,2014-02-04
4,5,7,25,6,2014-02-04
...,...,...,...,...,...
933,934,981,1508,2,2020-12-27
934,935,979,1509,6,2020-12-29
935,936,976,1510,6,2020-12-30
936,937,984,1511,2,2020-12-30


In [354]:
sell_prices = []
for i in car_ids_sell:
    b_price = df_buy[df_buy['id_samochodu'] == i]['cena'].values[0]
    print(b_price)
    sell_prices.append(np.round(b_price * rng.uniform(1.2, 1.5), -2))

df_sell['cena'] = sell_prices

25700.0
25700.0
32700.0
72400.0
45900.0
42500.0
49200.0
36500.0
4300.0
27900.0
64200.0
31900.0
27100.0
141200.0
35900.0
57800.0
25900.0
22500.0
121000.0
124600.0
29100.0
25600.0
21600.0
43200.0
21400.0
159000.0
50100.0
30100.0
90500.0
15100.0
29100.0
35700.0
36600.0
71100.0
14300.0
76700.0
30300.0
37300.0
43900.0
79500.0
58200.0
71300.0
34500.0
47800.0
31500.0
47600.0
15100.0
31200.0
23600.0
33500.0
48900.0
20500.0
44100.0
61900.0
61300.0
29500.0
138000.0
22200.0
67100.0
89000.0
29400.0
53700.0
38000.0
40400.0
216100.0
52400.0
100600.0
43700.0
44100.0
38200.0
71500.0
12500.0
24800.0
27100.0
14900.0
25000.0
42600.0
23900.0
114300.0
114300.0
119000.0
18900.0
104400.0
44300.0
51000.0
20800.0
14200.0
66500.0
65800.0
13400.0
70300.0
57600.0
34900.0
25800.0
89500.0
65400.0
81700.0
31700.0
42500.0
165000.0
31400.0
47000.0
10600.0
80500.0
30100.0
47300.0
94300.0
16700.0
50300.0
95700.0
216100.0
30200.0
17200.0
45600.0
53100.0
47800.0
53900.0
78900.0
23000.0
24900.0
26700.0
88500.0
55500.0
7190

In [355]:
df_sell

,id,id_samochodu,id_klienta,id_pracownika,data_zaksiegowania,cena
0,1,18,21,2,2014-01-30,32200.0
1,2,17,22,2,2014-01-30,35200.0
2,3,9,23,6,2014-02-04,40300.0
3,4,16,24,2,2014-02-04,95600.0
4,5,7,25,6,2014-02-04,62400.0
...,...,...,...,...,...,...
933,934,981,1508,2,2020-12-27,33800.0
934,935,979,1509,6,2020-12-29,38400.0
935,936,976,1510,6,2020-12-30,31800.0
936,937,984,1511,2,2020-12-30,46000.0


In [356]:
np.sum(df_sell["cena"] == float('inf')) # wychodzą inf wiec trzeba poprawić

0

# Użyte części

In [357]:
data_parts = get_data_parts()
data_parts_filtered = data_parts[data_parts['motor'] == True]

def get_parts_indexes(car_id):
    if df_samochody.loc[car_id-1].values[3] == 2:
        parts_set = data_parts_filtered[['id','część']]
    else:
        parts_set = data_parts[['id','część']]
    
    parts_indexes = parts_set['id']
    num_parts = rng.choice([1, 2, 3], p=[0.6, 0.3, 0.1])
    indexes = rng.choice(parts_indexes, size=num_parts)
    return indexes

all_car_parts_indexes = [(i, get_parts_indexes(i)) for i in car_ids_service]
print(all_car_parts_indexes)

[(21, array([54], dtype=int64)), (37, array([4], dtype=int64)), (42, array([51], dtype=int64)), (50, array([60], dtype=int64)), (60, array([9], dtype=int64)), (68, array([2], dtype=int64)), (71, array([51], dtype=int64)), (77, array([45, 18, 13], dtype=int64)), (79, array([ 3, 45], dtype=int64)), (89, array([28], dtype=int64)), (103, array([27, 51, 52], dtype=int64)), (111, array([45, 41, 56], dtype=int64)), (117, array([ 9, 60], dtype=int64)), (120, array([46], dtype=int64)), (121, array([48], dtype=int64)), (123, array([47], dtype=int64)), (127, array([38], dtype=int64)), (135, array([36, 27], dtype=int64)), (151, array([21, 38], dtype=int64)), (152, array([52,  5], dtype=int64)), (153, array([7], dtype=int64)), (157, array([15, 17], dtype=int64)), (192, array([27], dtype=int64)), (203, array([40, 36], dtype=int64)), (208, array([56, 31, 29], dtype=int64)), (212, array([21, 44], dtype=int64)), (215, array([26], dtype=int64)), (220, array([15,  2], dtype=int64)), (225, array([62, 31],

In [358]:
#df_samochody.loc()[1] - samochód o indeksie 0

def create_part(id, service_id, car):
    part = data_parts.loc()[id]
    if type(part['atrybuty']) != float:
        attributes = part['atrybuty'].split()
        for i in range(len(attributes)):
            if attributes[i] == 'marka':
                attributes[i] = car['marka']
            if attributes[i] == 'model':
                attributes[i] = car['model']
            if attributes[i] == 'kolor':
                attributes[i] = car['kolor']
            if attributes[i] == 'rozmiar':
                attributes[i] = 'R' + str(rng.choice([15, 16, 17]))
            if attributes[i] == 'material':
                attributes[i] = rng.choice(['skóra', 'materiał'], p=[0.2, 0.8])
            if attributes[i] == 'pojemnosc':
                attributes[i] = str(car['pojemność_silnika'])
            if attributes[i] == 'skrzynia':
                attributes[i] = str(car['skrzynia_biegów'])
        price = int(np.floor(part['cena']*car['pojemność_silnika']/1.8 * rng.uniform(0.8, 1.2)))
    else:
        attributes = ''
        price = part['cena']
    quantity = part['maxcount']
    return part['część'], ' '.join(attributes), price, service_id, quantity

parts = []
for service_id, element in enumerate(all_car_parts_indexes):
    car_id, parts_id = element
    car = df_samochody.loc()[car_id-1]
    for part_id in parts_id:
        parts.append(create_part(part_id, service_id+1, car))

parts = np.array(parts)
parts_df = pd.DataFrame(parts, columns=['nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']).drop_duplicates(['nazwa', 'atrybuty'])
parts_df['id'] = np.arange(1, len(parts_df)+1)
parts_df = parts_df[['id', 'nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']]

parts = pd.DataFrame(parts, columns=['nazwa', 'atrybut', 'cena', 'id_usługi', 'ilość'])

In [359]:
parts

,nazwa,atrybut,cena,id_usługi,ilość
0,Amortyzator,,150,1,4
1,Felga,R17 niebieski,104,2,4
2,Pompa paliwowa,Volkswagen Crafter,421,3,1
3,Dźwignia zmiany biegów,Peugeot 108,59,4,1
4,Drzwi prawe przednie,BMW 320 biały,712,5,1
...,...,...,...,...,...
170,Felga,R17 czerwony,62,117,4
171,Reflektor,Royal Enfield Thunderbird 350X,20,118,1
172,Ogniwo paliwowe,Opel Astra,420,119,1
173,Pompa paliwowa,Bajaj Pulsar 150,54,120,1


# Wyposażenie

In [360]:
eq_len = len(parts_df)
equipment_id = parts_df['id']
equipment_name = parts_df['nazwa']
equipment_attr = parts_df['atrybuty']
equipment_price = parts_df['cena']

df_equipment = pd.DataFrame()
df_equipment['id'] = equipment_id
df_equipment['nazwa'] = equipment_name
df_equipment['atrybut'] = equipment_attr
df_equipment['cena_części'] = equipment_price.astype(float)

In [361]:
df_equipment

,id,nazwa,atrybut,cena_części
0,1,Amortyzator,,150.0
1,2,Felga,R17 niebieski,104.0
2,3,Pompa paliwowa,Volkswagen Crafter,421.0
3,4,Dźwignia zmiany biegów,Peugeot 108,59.0
4,5,Drzwi prawe przednie,BMW 320 biały,712.0
...,...,...,...,...
169,142,Klocek hamulcowy,Volvo V70,30.0
170,143,Felga,R17 czerwony,62.0
171,144,Reflektor,Royal Enfield Thunderbird 350X,20.0
172,145,Ogniwo paliwowe,Opel Astra,420.0


# Usługi - uzupełnienie

In [362]:
service_prices = np.zeros(len(service_ids))
for i in range(len(parts)):
    row = parts.loc()[i]
    service_index = int(row['id_usługi'])
    service_prices[service_index-1] += float(row['cena']) * int(row['ilość'])

df_services['cena'] = np.round(service_prices*rng.uniform(1.1, 1.4), -1).astype(int)

In [363]:
df_services

,id,id_samochodu,id_klienta,id_pracownika,data_zaksiegowania,cena
0,1,21,38,9,2014-02-12,820
1,2,37,69,8,2014-03-21,570
2,3,42,78,9,2014-04-03,580
3,4,50,51,7,2014-04-29,80
4,5,60,111,7,2014-05-18,980
...,...,...,...,...,...,...
116,117,289,505,9,2020-09-23,340
117,118,961,1479,7,2020-10-30,30
118,119,971,1493,7,2020-12-02,580
119,120,975,1260,4,2020-12-06,70


# Użyte części - uzupełnienie

In [364]:
used_parts_df = parts.merge(df_equipment, on=['nazwa', 'atrybut'])
used_parts_df.rename(columns={'id': 'id_części'}, inplace=True)
used_parts_df['ilość'] = pd.to_numeric(used_parts_df['ilość'])
ids = np.arange(len(used_parts_df))+1
used_parts_df['id'] = ids
used_parts_df = used_parts_df[['id', 'id_usługi', 'id_części', 'ilość']]

In [365]:
used_parts_df

,id,id_usługi,id_części,ilość
0,1,1,1,4
1,2,2,2,4
2,3,3,3,1
3,4,4,4,1
4,5,5,5,1
...,...,...,...,...
170,171,117,143,4
171,172,118,144,1
172,173,119,145,1
173,174,120,146,1


# Zakup części

In [366]:
def get_service_date(part_id):
    
    index_used_part = used_parts_df[used_parts_df['id_części'] == part_id].index[0]
    service_date = df_services[df_services['id'] == index_used_part]['data_zaksiegowania']
    return service_date
    

buy_parts_quantity = used_parts_df.groupby(['id_części'])['ilość'].sum().reset_index()
parts_purchase_pd = pd.DataFrame(columns=['id', 'id_części', 'koszt', 'data_zakupu', 'ilość', 'data_zaksięgowania'])

for i in range(len(buy_parts_quantity)):
    part_id = buy_parts_quantity['id_części'][i]
    quantity = buy_parts_quantity['ilość'][i]
    price = df_equipment['cena_części'][df_equipment['id'] == part_id].to_numpy()
    if price > 1000:
        quantity += rng.random() < 0.5
    elif price > 500:
        quantity += rng.choice([0, 1, 2], p=[0.2, 0.5, 0.3])
    else:
        quantity += rng.binomial(25, 0.15)
    total_cost = np.floor(quantity * price * rng.uniform(0.75, 0.95))
    service_date = get_service_date(part_id)

    ##### DO DODANIA
    # generowanie daty zakupu części na podstawie purchase_date
    # generowanie daty zaksięgowania transakcji na podstawie purchase_date
    #####
    
    # row = [i+1, part_id, total_cost, purchase_date, quantity, posting_date]
    # parts_purchase_pd.loc[i] = row


In [367]:
# parts_purchase_pd